## Data Preprocessing

In [10]:
import pandas as pd 
df = pd.read_csv("../Data/lstm.csv")
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [11]:
df.category.unique()

array(['food', 'sports'], dtype=object)

In [12]:
# 데이터에 사용된 중복 없는 전체 단어 갯수 파악 
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size

536

In [13]:
# 단어를 숫자로 인코딩 
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [14]:
from tensorflow import keras 
keras.utils.set_random_seed(1) # keras에서 set seed 하기 

In [15]:
# One Hot Encoding (단어로 되어있는 걸 숫자로 바꾸기)
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
encoded_paragraphs

[[144,
  429,
  167,
  458,
  207,
  175,
  360,
  429,
  139,
  223,
  378,
  420,
  9,
  520,
  438,
  335,
  382,
  20,
  50,
  255,
  302,
  100,
  258,
  212,
  433,
  135,
  274,
  84,
  20,
  69,
  212,
  347,
  86,
  85,
  48,
  87,
  302,
  400,
  130,
  65,
  356,
  274,
  336,
  35,
  126,
  504,
  25,
  453,
  162,
  378,
  457,
  20,
  475],
 [183,
  438,
  335,
  77,
  255,
  65,
  258,
  9,
  417,
  287,
  404,
  393,
  135,
  48,
  416,
  190,
  460,
  489,
  12],
 [260,
  282,
  192,
  78,
  321,
  530,
  135,
  417,
  7,
  482,
  282,
  130,
  336,
  417,
  321,
  530,
  429,
  84,
  165,
  477,
  167,
  364,
  295,
  61,
  26,
  460,
  212,
  20,
  473,
  395,
  453,
  282,
  59,
  135,
  526,
  392,
  20,
  475,
  165,
  154,
  347,
  48,
  450],
 [480,
  176,
  409,
  10,
  34,
  378,
  377,
  135,
  10,
  84,
  135,
  377,
  417,
  378,
  404,
  400,
  175,
  135,
  378,
  417,
  400,
  336,
  412,
  417,
  19,
  30,
  378,
  417,
  161,
  162,
  143,
  261,
  9,


In [18]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인 (최대값 찾는 로직)
max_length = 0 
for row in df['paragraph'] :
    if len(row.split(" "))  > max_length :
        max_length = len(row.split(" "))
print(max_length)

91


In [19]:
# zero padding(sequence padding) 넣어서 문장별 길이 맞추기 
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(encoded_paragraphs, maxlen = max_length, padding = 'post')
# - padding :'post' - 0값 뒤로 빼는 것 

padded_paragraphs_encoding

array([[144, 429, 167, ...,   0,   0,   0],
       [183, 438, 335, ...,   0,   0,   0],
       [260, 282, 192, ...,   0,   0,   0],
       ...,
       [455, 385, 347, ...,   0,   0,   0],
       [  9, 378, 192, ...,   0,   0,   0],
       [428,  84, 378, ...,   0,   0,   0]], dtype=int32)

In [20]:
# 분류항목(food, sports)를 수치로 변경
categories = df['category'].to_list()

def category_encode(category) :
    if category == "food":
        return [1,0]
    else : 
        return [0,1]

In [21]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [22]:
# Feature 확인 
print(encoded_paragraphs[0])

[144, 429, 167, 458, 207, 175, 360, 429, 139, 223, 378, 420, 9, 520, 438, 335, 382, 20, 50, 255, 302, 100, 258, 212, 433, 135, 274, 84, 20, 69, 212, 347, 86, 85, 48, 87, 302, 400, 130, 65, 356, 274, 336, 35, 126, 504, 25, 453, 162, 378, 457, 20, 475]


In [23]:
len(encoded_paragraphs[19])

73

--- 
## RNN Model

In [24]:
model = keras.Sequential()
# 문맥 생성 단계 

model.add(keras.layers.Embedding(vocab_size, 5, input_length = max_length))
# Embedding : One Hot Encoding 하기 (지금 다 숫자로 되어있으니까)
# vocab_size : 전체 데이터 피쳐 갯수 
# 5 : embedding layer(인덱스를 받아 5차원 벡터의 임베딩을 출력). Hyper Parameter. 
# ---> 단어 하나를 5개 차원으로 나눠서 임베딩 해보겠다는 것 (근데 그냥 5개 품사 정도로 크게 나눠서 (5형식?) 그걸로 한거임. 컴퓨터는 뭔 의민지 당연히 모르고)
# max_length : 한 문장당 단어 최대 갯수 (아까 맞춘거.)

# LSTM 
model.add(keras.layers.LSTM(64)) 

# 분류단계 
model.add(keras.layers.Dense(32, activation = 'relu')) # LSTM 반값 쓰기 
model.add(keras.layers.Dense(2, activation = 'softmax')) # 문장의 확률값을 구하고자 함 

2022-07-22 14:19:59.116487: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# 컴파일 
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [26]:
# 데이터셋 리스트 -> 넘파이 배열 변환 (길이 맞췄으니까 이제 넘파이 사용 간ㅇ)
import numpy as np 
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [27]:
# 학습하기 
model.fit(train_X, train_Y, batch_size=10, epochs=50)

Epoch 1/50
2/2 [==============================] - 2s 23ms/step - loss: 0.6939 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6923 - accuracy: 0.5500
Epoch 3/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6921 - accuracy: 0.6000
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.6919 - accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6916 - accuracy: 0.4000
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 0.6910 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.6903 - accuracy: 0.5000
Epoch 8/50
2/2 [==============================] - 0s 20ms/step - loss: 0.6895 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6887 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6881 - accuracy: 0.6000
Epoch 11/50
2/2 [============